### Adding homeport from vessel 

In [3]:
import pandas as pd
import xgboost as xgb
import numpy as np


In [4]:
train = pd.read_csv('data/datasets/ais_train.csv', sep='|')
train['time'] = pd.to_datetime(train['time'])
train.head()

test = pd.read_csv('data/datasets/ais_test.csv', sep=',')
test['time'] = pd.to_datetime(test['time'])
test.head()

vessels = pd.read_csv('data/datasets/vessels.csv', sep='|')
vessels.head()

ports = pd.read_csv('data/datasets/ports.csv', sep='|')
ports.head

<bound method NDFrame.head of                         portId               name portLocation   longitude  \
0     61d36ed80a1807568ff9a064    Port of Algiers      Algiers    3.067222   
1     61d36ed80a1807568ff9a065     Port of Annaba       Annaba    7.772500   
2     61d36edf0a1807568ff9a070       Port of Oran         Oran   -0.639722   
3     61d36ee00a1807568ff9a072     Port of Skikda       Skikda    6.905833   
4     61d36ee10a1807568ff9a074  Port of Pago-Pago    Pago-Pago -170.690556   
...                        ...                ...          ...         ...   
1324  6367c622aa0cd188707a1eae              Hamad        Hamad   25.029460   
1325  6367cb14aa0cd188707a1eaf            Al Hidd      Al Hidd   50.667890   
1326  6367d9afaa0cd188707a1eb3             Aquaba       Aquaba   34.992790   
1327  663cb287244f580d97afce53   Port of Portbury     Portbury   -2.648735   
1328  663cb48be894e2f07d1c69b2       Bahrain Port      Bahrain   50.616667   

       latitude UN_LOCODE     cou

In [5]:
port_id_mapping = {port_id: idx for idx, port_id in enumerate(train['portId'].unique())}
train['portId'] = train['portId'].map(port_id_mapping)
ports['portId'] = ports['portId'].map(port_id_mapping)

vessel_id_mapping = {vessel_id: idx for idx, vessel_id in enumerate(train['vesselId'].unique())}
train['vesselId'] = train['vesselId'].map(vessel_id_mapping)
vessels['vesselId'] = vessels['vesselId'].map(vessel_id_mapping)
test['vesselId'] = test['vesselId'].map(vessel_id_mapping)

shipping_line_id_mapping = {shipping_line_id: idx for idx, shipping_line_id in enumerate(vessels['shippingLineId'].unique())}
vessels['shippingLineId'] = vessels['shippingLineId'].map(shipping_line_id_mapping)

In [6]:
vessels.head()

,shippingLineId,vesselId,CEU,DWT,GT,NT,vesselType,breadth,depth,draft,enginePower,freshWater,fuel,homePort,length,maxHeight,maxSpeed,maxWidth,rampCapacity,yearBuilt
0,0,599.0,6500,21200.0,58684,17606.0,83.0,32.0,22.20,NaN,0.0,NaN,NaN,OSLO,199.00,5.0,18.6,15.2,150.0,2000
1,1,65.0,4902,12325.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,MONROVIA,182.00,NaN,NaN,NaN,NaN,2006
2,2,640.0,5000,13059.0,46800,NaN,83.0,31.0,NaN,NaN,14220.0,NaN,NaN,SAINT JOHN'S,182.00,NaN,NaN,NaN,NaN,2010
3,3,255.0,4200,12588.0,39362,NaN,83.0,28.0,NaN,NaN,11060.0,NaN,NaN,NaN,167.00,NaN,NaN,NaN,NaN,2011
4,4,68.0,7450,21052.0,75528,24391.0,83.0,37.2,22.23,NaN,13140.0,491.47,3236.78,Panama,199.98,NaN,NaN,NaN,NaN,2018


In [7]:
train = train.merge(ports[['portId', 'latitude', 'longitude']], how='left', left_on='portId', right_on='portId', suffixes=('', '_port'))
train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,latitude_port,longitude_port
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,0,0,-33.5875,-71.618889
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,1,1,8.9670,-79.533000
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,2,2,39.2325,-76.558889
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,3,3,-34.4625,150.899444
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,4,4,35.7830,-5.817000


In [8]:


# Count the frequency of each port visit for each vessel
port_visits = train.groupby(['vesselId', 'portId']).size().reset_index(name='visit_count')

# Sort by vesselId and visit_count to see the most visited ports
sorted_port_visits = port_visits.sort_values(by=['vesselId', 'visit_count'], ascending=[True, False])

favorite_ports = sorted_port_visits.groupby('vesselId').first().reset_index()
favorite_ports = favorite_ports[['vesselId', 'portId']]
favorite_ports.columns = ['vesselId', 'favorite_port']

# Merge favorite_ports with train based on vesselId
train = train.merge(favorite_ports, on='vesselId', how='left')
# Display the result
train.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,latitude_port,longitude_port,favorite_port
0,2024-01-01 00:00:25,284.0,0.7,0,88,0,01-09 23:00,-34.74370,-57.85130,0,0,-33.5875,-71.618889,170
1,2024-01-01 00:00:36,109.6,0.0,-6,347,1,12-29 20:00,8.89440,-79.47939,1,1,8.9670,-79.533000,77
2,2024-01-01 00:01:45,111.0,11.0,0,112,0,01-02 09:00,39.19065,-76.47567,2,2,39.2325,-76.558889,2
3,2024-01-01 00:03:11,96.4,0.0,0,142,1,12-31 20:00,-34.41189,151.02067,3,3,-34.4625,150.899444,8
4,2024-01-01 00:03:51,214.0,19.7,0,215,0,01-25 12:00,35.88379,-5.91636,4,4,35.7830,-5.817000,2


In [17]:
train = train.merge(vessels[['vesselId', 'length', 'shippingLineId']], on='vesselId', how='left')
train['vessel_type'] = np.where(train['length'] > 200, 'deep-sea', 'short-sea')
train.head()


KeyError: 'length'

In [10]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a)) 
    r = 6371  # Radius of Earth in kilometers
    return r * c  # Distance in kilometers

### Feature engineering 

In [11]:
def feature_engineering(train):
    train = train.copy()
    features = pd.DataFrame()

    # Add the columns vesselId, time, latitude, longitude, cog and sog to the features from train
    features['vesselId'] = train['vesselId']
    features['time'] = train['time']
    features['latitude'] = train['latitude']
    features['longitude'] = train['longitude']
    features['latitude_port'] = train['latitude_port']
    features['longitude_port'] = train['longitude_port']    
    features['cog'] = train['cog']
    features['sog'] = train['sog']

    # Create a new binary column for 'under_way' based on the navstat column in train. 
    # 1 if navstat is 0 or 8, 0 otherwise
    features['under_way'] = train['navstat'].isin([0, 8]).astype(int)

    # Adding the columns latitude_1 (last seen latitude) and longitude_1 (last senn longitude)
    # to the features from train. For each vessel
    # Also adding time_1 (time difference from current time to last seen time)
    features['latitude_1'] = train.groupby('vesselId')['latitude'].shift(1)
    features['longitude_1'] = train.groupby('vesselId')['longitude'].shift(1)
    features['time_1'] = train.groupby('vesselId')['time'].diff().dt.total_seconds()

    # Adding the columns latitude_2 (second last seen latitude) and longitude_2 (second last seen longitude)
    # to the features from train. For each vessel
    # Also adding time_2 (time difference from current time to second last seen time)
    features['latitude_2'] = train.groupby('vesselId')['latitude'].shift(2)
    features['longitude_2'] = train.groupby('vesselId')['longitude'].shift(2)
    features['time_2'] = train.groupby('vesselId')['time'].diff(2).dt.total_seconds()

    # Adding the columns latitude_3 (third last seen latitude) and longitude_3 (third last seen longitude)
    # to the features from train. For each vessel
    # Also adding time_3 (time difference from current time to third last seen time)
    features['latitude_3'] = train.groupby('vesselId')['latitude'].shift(3)
    features['longitude_3'] = train.groupby('vesselId')['longitude'].shift(3)
    features['time_3'] = train.groupby('vesselId')['time'].diff(3).dt.total_seconds()


    # Dropping the rows with NaN values
    features = features.dropna()

    # Split the time column into month, day, hour, minute and second columns
    features['month'] = train['time'].dt.month
    features['day'] = train['time'].dt.day
    features['hour'] = train['time'].dt.hour
    features['minute'] = train['time'].dt.minute
    features['second'] = train['time'].dt.second

   
    
    features['distance_to_port'] = haversine(features['latitude'], features['longitude'], features['latitude_port'], features['longitude_port'])
    # Group by vesselId and time to ensure the features are calculated correctly
    features = features.groupby(['vesselId', 'time']).first().reset_index()
    features['rolling_distance_mean'] = features.groupby('vesselId')['distance_to_port'].transform(lambda x: x.rolling(window=6, min_periods=1).mean())
    features['rolling_distance_sum'] = features.groupby('vesselId')['distance_to_port'].transform(lambda x: x.rolling(window=6, min_periods=1).sum())

    

    # Create a flag if the rolling distance has been increasing over the last 6 values
    features['rolling_distance_increasing'] = features.groupby('vesselId')['rolling_distance_mean'].transform(lambda x: x.rolling(window=6).apply(lambda y: all(y[i] < y[i+1] for i in range(len(y)-1)), raw=True)).fillna(0).astype(int)
    # features.drop('time', axis=1, inplace=True)
    # # Create rolling window features for distance to port


    features.drop('time', axis=1, inplace=True)

    return features

features = feature_engineering(train)
print(features.head())
print(features.shape)


   vesselId  latitude  longitude  latitude_port  longitude_port   cog   sog  \
0         0 -35.16805  -56.53190       -33.5875      -71.618889  88.2  14.3   
1         0 -35.16715  -56.45306       -33.5875      -71.618889  88.3  12.3   
2         0 -35.16646  -56.40306       -33.5875      -71.618889  86.8  11.7   
3         0 -35.16544  -56.23866       -33.5875      -71.618889  70.4  15.2   
4         0 -35.08926  -55.50466       -33.5875      -71.618889  91.1  15.0   

   under_way  latitude_1  longitude_1  ...   time_3  month  day  hour  minute  \
0          1   -35.16863    -56.63185  ...  25110.0      1    1     6      58   
1          1   -35.16805    -56.53190  ...   4008.0      1    1     7      15   
2          1   -35.16715    -56.45306  ...   2996.0      1    1     7      28   
3          1   -35.16646    -56.40306  ...   3880.0      1    1     8       3   
4          1   -35.16544    -56.23866  ...  12823.0      1    1    10      49   

   second  distance_to_port  rolling_d

In [12]:
features.head()

,vesselId,latitude,longitude,latitude_port,longitude_port,cog,sog,under_way,latitude_1,longitude_1,...,time_3,month,day,hour,minute,second,distance_to_port,rolling_distance_mean,rolling_distance_sum,rolling_distance_increasing
0,0,-35.16805,-56.53190,-33.5875,-71.618889,88.2,14.3,1,-35.16863,-56.63185,...,25110.0,1,1,6,58,55,1394.309141,1394.309141,1394.309141,0
1,0,-35.16715,-56.45306,-33.5875,-71.618889,88.3,12.3,1,-35.16805,-56.53190,...,4008.0,1,1,7,15,56,1401.461476,1397.885308,2795.770617,0
2,0,-35.16646,-56.40306,-33.5875,-71.618889,86.8,11.7,1,-35.16715,-56.45306,...,2996.0,1,1,7,28,15,1405.997112,1400.589243,4201.767729,0
3,0,-35.16544,-56.23866,-33.5875,-71.618889,70.4,15.2,1,-35.16646,-56.40306,...,3880.0,1,1,8,3,35,1420.918535,1405.671566,5622.686264,0
4,0,-35.08926,-55.50466,-33.5875,-71.618889,91.1,15.0,1,-35.16544,-56.23866,...,12823.0,1,1,10,49,39,1487.286024,1421.994458,7109.972288,0


In [13]:
def find_last_features(features):
    # Assuming that the last row for every vesselId is the most recent
    last_features = features.groupby('vesselId').last().reset_index()
    return last_features

last_features = find_last_features(features)
print(last_features.head())
print(last_features.shape)

   vesselId  latitude  longitude  latitude_port  longitude_port    cog   sog  \
0         0  34.57936  128.99926      35.109444      129.059444  221.5  15.5   
1         1   1.24460  103.39997      51.498889       -2.712222  305.1  15.7   
2         2  18.13873  -69.74863      18.475278      -69.881111  176.0   0.4   
3         3  41.64055  143.29942      42.988611      144.353611   87.6  14.4   
4         4  26.58710  121.27831      27.583000      120.450000   39.1  12.7   

   under_way  latitude_1  longitude_1  ...   time_3  month  day  hour  minute  \
0          1    34.59684    129.01917  ...   2795.0      5    1    12      41   
1          1     1.22186    103.46804  ...   3431.0      4   30    10      12   
2          1    18.14185    -69.74807  ...   3710.0      5    7    23      59   
3          1    41.63254    141.92751  ...  16296.0      4   26    13      29   
4          1    26.54636    121.23948  ...   4718.0      5    7    12      28   

   second  distance_to_port  rol

### Train the model

In [14]:
# # Define features and target
y = features[['latitude', 'longitude']]  
X = features.drop(columns=['latitude', 'longitude']) 

X.head()

,vesselId,latitude_port,longitude_port,cog,sog,under_way,latitude_1,longitude_1,time_1,latitude_2,...,time_3,month,day,hour,minute,second,distance_to_port,rolling_distance_mean,rolling_distance_sum,rolling_distance_increasing
0,0,-33.5875,-71.618889,88.2,14.3,1,-35.16863,-56.63185,1236.0,-35.16787,...,25110.0,1,1,6,58,55,1394.309141,1394.309141,1394.309141,0
1,0,-33.5875,-71.618889,88.3,12.3,1,-35.16805,-56.53190,1021.0,-35.16863,...,4008.0,1,1,7,15,56,1401.461476,1397.885308,2795.770617,0
2,0,-33.5875,-71.618889,86.8,11.7,1,-35.16715,-56.45306,739.0,-35.16805,...,2996.0,1,1,7,28,15,1405.997112,1400.589243,4201.767729,0
3,0,-33.5875,-71.618889,70.4,15.2,1,-35.16646,-56.40306,2120.0,-35.16715,...,3880.0,1,1,8,3,35,1420.918535,1405.671566,5622.686264,0
4,0,-33.5875,-71.618889,91.1,15.0,1,-35.16544,-56.23866,9964.0,-35.16646,...,12823.0,1,1,10,49,39,1487.286024,1421.994458,7109.972288,0


In [15]:
# Initialize the xgboost model
model = xgb.XGBRegressor()

# Fit the model
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [296]:
X.head()

,vesselId,latitude_port,longitude_port,cog,sog,under_way,latitude_1,longitude_1,time_1,latitude_2,...,time_3,month,day,hour,minute,second,distance_to_port,rolling_distance_mean,rolling_distance_sum,rolling_distance_increasing
0,0,-33.5875,-71.618889,88.2,14.3,1,-35.16863,-56.63185,1236.0,-35.16787,...,25110.0,1,1,6,58,55,1394.309141,1394.309141,1394.309141,0
1,0,-33.5875,-71.618889,88.3,12.3,1,-35.16805,-56.53190,1021.0,-35.16863,...,4008.0,1,1,7,15,56,1401.461476,1397.885308,2795.770617,0
2,0,-33.5875,-71.618889,86.8,11.7,1,-35.16715,-56.45306,739.0,-35.16805,...,2996.0,1,1,7,28,15,1405.997112,1400.589243,4201.767729,0
3,0,-33.5875,-71.618889,70.4,15.2,1,-35.16646,-56.40306,2120.0,-35.16715,...,3880.0,1,1,8,3,35,1420.918535,1405.671566,5622.686264,0
4,0,-33.5875,-71.618889,91.1,15.0,1,-35.16544,-56.23866,9964.0,-35.16646,...,12823.0,1,1,10,49,39,1487.286024,1421.994458,7109.972288,0


,vesselId,latitude_port,longitude_port,cog,sog,under_way,latitude_1,longitude_1,time_1,latitude_2,...,time_3,month,day,hour,minute,second,distance_to_port,rolling_distance_mean,rolling_distance_sum,rolling_distance_increasing
0,0,-33.5875,-71.618889,88.2,14.3,1,-35.16863,-56.63185,1236.0,-35.16787,...,25110.0,1,1,6,58,55,1394.309141,1394.309141,1394.309141,0
1,0,-33.5875,-71.618889,88.3,12.3,1,-35.16805,-56.53190,1021.0,-35.16863,...,4008.0,1,1,7,15,56,1401.461476,1397.885308,2795.770617,0
2,0,-33.5875,-71.618889,86.8,11.7,1,-35.16715,-56.45306,739.0,-35.16805,...,2996.0,1,1,7,28,15,1405.997112,1400.589243,4201.767729,0
3,0,-33.5875,-71.618889,70.4,15.2,1,-35.16646,-56.40306,2120.0,-35.16715,...,3880.0,1,1,8,3,35,1420.918535,1405.671566,5622.686264,0
4,0,-33.5875,-71.618889,91.1,15.0,1,-35.16544,-56.23866,9964.0,-35.16646,...,12823.0,1,1,10,49,39,1487.286024,1421.994458,7109.972288,0


In [289]:
### Prepare test data for predictions
def prepare_test_for_predictions(test, last_features):
    test = test.copy()
    prepared_test = pd.DataFrame()

    # Create a time column in last test
    last_features['year'] = 2024
    print(last_features.columns)
    last_features['time'] = pd.to_datetime(
        last_features[['year', 'month', 'day', 'hour', 'minute', 'second']]
    )
    last_features =last_features.drop(columns=['longitude_3', 'latitude_3', 'year', 'month', 'day', 'hour', 'minute', 'second', 'time_3']).copy()
    print(last_features.columns)

    # Add the columns vesselId and time
    prepared_test['vesselId'] = test['vesselId']
    prepared_test['time'] = test['time']

    # For each vessel, add the last seen to the prepared test
    prepared_test = prepared_test.merge(last_features, on='vesselId', how='left', suffixes=('', '_last'))
    print(prepared_test)
    print(test.columns)

    # Add the empty columns latitude_3, longitude_3 and time_3 to the prepared test
    prepared_test['latitude_3'] = None
    prepared_test['longitude_3'] = None
    prepared_test['time_3'] = None

    # For each row, latitude_3 should be the same as latitude_2, longitude_3 should be the same as longitude_2
    # and time_3 should be the same as time_2
    prepared_test['latitude_3'] = prepared_test['latitude_2']
    prepared_test['longitude_3'] = prepared_test['longitude_2']
    prepared_test['time_3'] = prepared_test['time_2']

    # Same for latitude_2, longitude_2 and time_2
    prepared_test['latitude_2'] = prepared_test['latitude_1']
    prepared_test['longitude_2'] = prepared_test['longitude_1']
    prepared_test['time_2'] = prepared_test['time_1']

    # Same for latitude_1, longitude_1 and time_1
    prepared_test['latitude_1'] = prepared_test['latitude']
    prepared_test['longitude_1'] = prepared_test['longitude']

    # Time_1 should be the difference between the current time and the last seen time
    prepared_test['time_1'] = (prepared_test['time'] - prepared_test['time_last']).dt.total_seconds()

    # For each time column (time_2, time_3), add the rows value in time_1
    prepared_test['time_2'] += prepared_test['time_1']
    prepared_test['time_3'] += prepared_test['time_1']

    # Drop the the latitude and longitude columns
    prepared_test.drop(columns=['latitude', 'longitude'], inplace=True)
    print(prepared_test.shape)

    # Split the time column into month, day, hour, minute and second columns
    prepared_test['month'] = test['time'].dt.month
    prepared_test['day'] = test['time'].dt.day
    prepared_test['hour'] = test['time'].dt.hour
    prepared_test['minute'] = test['time'].dt.minute
    prepared_test['second'] = test['time'].dt.second
    

    prepared_test.drop('time', axis=1, inplace=True)
    prepared_test.drop('time_last', axis=1, inplace=True)

    return prepared_test


# Prepare test data for predictions
test_df = prepare_test_for_predictions(test, last_features)
print(test_df.head())
print(test_df.shape)


Index(['vesselId', 'latitude', 'longitude', 'latitude_port', 'longitude_port',
       'cog', 'sog', 'under_way', 'latitude_1', 'longitude_1', 'time_1',
       'latitude_2', 'longitude_2', 'time_2', 'latitude_3', 'longitude_3',
       'time_3', 'month', 'day', 'hour', 'minute', 'second',
       'distance_to_port', 'rolling_distance_mean', 'rolling_distance_sum',
       'rolling_distance_increasing', 'year'],
      dtype='object')
Index(['vesselId', 'latitude', 'longitude', 'latitude_port', 'longitude_port',
       'cog', 'sog', 'under_way', 'latitude_1', 'longitude_1', 'time_1',
       'latitude_2', 'longitude_2', 'time_2', 'distance_to_port',
       'rolling_distance_mean', 'rolling_distance_sum',
       'rolling_distance_increasing', 'time'],
      dtype='object')
       vesselId                time  latitude  longitude  latitude_port  \
0           412 2024-05-08 00:03:16  31.14647  -81.49789      31.140556   
1           373 2024-05-08 00:06:17  14.81694  120.29625      14.808333   

In [290]:

# Reorder the DataFrame to match the model's expected feature order
required_features = [
    'vesselId', 'latitude_port', 'longitude_port', 'cog', 'sog',
    'under_way', 'latitude_1', 'longitude_1', 'time_1', 
    'latitude_2', 'longitude_2', 'time_2', 'latitude_3', 
    'longitude_3', 'time_3', 'month', 'day', 'hour', 
    'minute', 'second', 'distance_to_port', 
    'rolling_distance_mean', 'rolling_distance_sum', 
    'rolling_distance_increasing'  # Now included
]

# Ensure the columns are in the correct order
test_df = test_df[required_features]

# Make predictions
predictions = model.predict(test_df)
print(predictions)

[[ 31.11087   -81.24903  ]
 [ 14.655463  119.953255 ]
 [ 38.295822   10.9198475]
 ...
 [ 33.67234   147.35547  ]
 [ 56.21705    15.068819 ]
 [ 39.287926  -14.771328 ]]


In [295]:
# Create a DataFrame with the required format
predictions_df = pd.DataFrame(predictions, columns=['latitude_predicted', 'longitude_predicted'])
predictions_df['ID'] = range(len(predictions_df))
predictions_df = predictions_df[['ID', 'longitude_predicted', 'latitude_predicted']]

# Save to CSV
predictions_df.to_csv('data/datasets/predictions_4.csv', index=False)